In [1]:
from StartingCandidateGenerator import StartingCandidateGenerator as SCG
from DataBaseInterface import DataBaseInterface as DBI
from CrossOperation import CrossOperation as CO
"Import necessary to fix atoms in the slab"
from ase.constraints import FixAtoms
from ase import Atoms
from ase.io import read
from generate_ceo2 import create_ceo2_111 as ceo2Slab
import numpy as np
from ase.visualize import view
from ase.calculators.emt import EMT
from ase.db import connect
from pathlib import Path
from ase.ga.startgenerator import StartGenerator

#--------------------------Define the fitness fucntion for your atoms-----------------------"

def fitness_function(atoms,env)-> float:
    o2_energy = read('test_data/o2_pw91.traj').get_potential_energy()
    reference=0.0# read('test_data/pt6o0_fs_meta.traj@:')[0].get_potential_energy()

    at_num= atoms.get_atomic_numbers()
    o_num = np.count_nonzero(at_num == 8)
    ce_num= np.count_nonzero(at_num == 58)

    o_num = o_num - (2* ce_num)
    fre = atoms.get_potential_energy() - reference - o_num * (0.5 * o2_energy) - o_num*env*0.5
    return fre

def relax_candidate(dbid,calculator):
    atoms = db.get_atoms_from_id(dbid)
    
    atoms.calc = calculator
    atoms.get_potential_energy()

    atoms.info['key_value_pairs']['raw_score'] = fitness_function(atoms,env)
    db.update_to_relaxed(dbid,atoms)

#---------------------------Generate static part of the system------------------------------"
slab = ceo2Slab(number_of_layers=1,repetitions = (5,5,1),vacuum=12.0)
a = 12.0
slab = Atoms('C',cell=[a,a,a],
             pbc=True)
c = FixAtoms(indices=np.arange(len(slab)))
slab.set_constraint(c)
#If slab atoms arent fixed they will be generated  for all structures, but will be relaxed!"

#---------------------------Generate constant part of the system------------------------------"
#Part of the system that can be relaxed but that doe snot vary in number over the duration of the search"
constant = Atoms('Pt6')

#---------Generate variable part of the system-(Single atom types only)------------------"
#Part of the system that can be relaxed and that varyies in number over the duration of the search"
#
variable_type = Atoms('O')
#Considered number of variable type atoms n the search"
variable_range = [0,1,2,3,4,7,8,9,10]

#---------------------------Define starting populatio--------------------------------"

candidateGenerator = SCG(slab,constant,variable_type,variable_range)
crossing = CO(slab,constant,variable_type,variable_range)
db = DBI('databaseGA.db')
population = 25

starting_pop = candidateGenerator.get_starting_population(population_size=population)



for i in starting_pop:
    db.add_unrelaxed_candidate(i )

        
#-----------------------------------Define Calculator----------------------------------------"

calc = EMT()

#---------------------------------Relax initial structures-----------------------------------"
env = -0.5 #Environment chem pot ox
while db.get_number_of_unrelaxed_candidates() > 0:

    atoms = db.get_first_unrelaxed()
    
    atoms.calc = calc
    atoms.get_potential_energy()
    
    atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env)
   
    db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)

#--------------------------------------Find better stoich to srtart eval----------------------------
#Overall fitness value
steps = 100
for i in range(steps):

    atomslist = db.get_better_candidates(n=20)
    "Mutate these atoms"
    crossing.cross(atomslist[0],atomslist[1])
    

ValueError: Array "positions" has wrong shape (1, 3, 3) != (1, 3).